In [1]:
import pandas as pd
import psycopg2 as pg

In [5]:
participant = pd.read_csv('C:/Users/DarkK/Documents/data_sci/ds004515/participants.tsv' , sep='\t')

participant_id  Original_ID sex  age GROUP  BDI  AUDIT  EDUCATION
0         sub-001        10600   F   43   CTL   14      1         16
1         sub-002        10601   F   50   CTL    2      2         16
2         sub-003        10602   F   22   CTL    2      1         18
3         sub-004        10603   F   18   CTL    0      1         13
4         sub-005        10604   M   31   CTL    1      1         14
5         sub-006        10605   F   24   CTL    8      3         19
6         sub-007        10606   M   44   ALC    5      7         16
7         sub-008        10607   M   38   ALC   42     27         13
8         sub-009        10608   M   32   ALC    1     11         16
9         sub-010        10609   F   54   ALC   17      9         12
10        sub-011        10610   F   37   ALC    5     15         18
11        sub-012        10611   M   51   ALC    1      7         16
12        sub-013        10612   F   42   ALC    0      7         16
13        sub-014        10613   M   33   ALC   24     19         18
14        sub-015        10614   M   50   ALC    0      7         12
15        sub-016        10615   M   28   ALC   40     21         16
16        sub-017        10618   M   36   ALC    6     10         17
17        sub-018        10619   F   44   ALC    0      5         10
18        sub-019        10620   F   27   CTL    2      1         18
19        sub-020        10621   F   34   CTL    1      1         16
20        sub-021        10622   M   49   ALC   22     10         16
21        sub-022        10623   F   49   ALC    3      3         12
22        sub-023        10624   F   30   ALC   24     11         16
23        sub-024        10625   F   29   ALC    5      6         12
24        sub-025        10626   M   44   ALC    5      9         16
25        sub-026        10627   F   28   ALC    0     13         13
26        sub-027        10628   M   45   ALC   34     11         18
27        sub-028        10629   F   37   ALC    4     12         18
28        sub-029        10630   F   44   ALC    5     11         12
29        sub-030        10631   F   45   ALC    8      7         17
30        sub-031        10632   F   25   ALC   14      5         15
31        sub-032        10633   F   55   ALC    6      5         12
32        sub-033        10634   F   24   ALC    3     10         16
33        sub-034        10635   F   34   ALC   17     20         16
34        sub-035        10636   M   30   ALC   10     10         16
35        sub-036        10637   F   31   CTL    4      1         22
36        sub-037        10638   F   34   CTL    3      0         13
37        sub-038        10639   F   24   ALC    8      5         16
38        sub-039        10640   M   22   CTL    0      3         12
39        sub-040        10642   F   27   CTL    2      0         17
40        sub-041        10643   M   33   CTL    8      0         15
41        sub-042        10644   F   26   CTL    0      1         21
42        sub-043        10645   F   50   CTL   37      2         16
43        sub-044        10646   F   32   CTL    0      1         14
44        sub-045        10647   M   47   CTL    2      3         16
45        sub-046        10648   M   40   CTL    2      0         16
46        sub-047        10649   F   42   CTL    0      0         23
47        sub-048        10650   M   48   CTL    2      1         16
48        sub-049        10651   F   50   CTL    5      2         16
49        sub-050        10652   M   43   CTL    0      2         18
50        sub-051        10653   F   53   CTL    0      2         18
51        sub-052        10656   M   44   CTL    7      0         12
52        sub-053        10657   M   47   CTL   16      2         12
53        sub-054        10658   M   52   CTL    1      2         12

In [4]:
connect = pg.connect(
    host='localhost',
    database='EEG_database',
    user='postgres',
    password='bank2739',
    port=5432
)

cursor = connect.cursor()

In [6]:
participant_id = participant['participant_id']
participant_group = participant['GROUP']


participant_id

participant_group

0     CTL
1     CTL
2     CTL
3     CTL
4     CTL
5     CTL
6     ALC
7     ALC
8     ALC
9     ALC
10    ALC
11    ALC
12    ALC
13    ALC
14    ALC
15    ALC
16    ALC
17    ALC
18    CTL
19    CTL
20    ALC
21    ALC
22    ALC
23    ALC
24    ALC
25    ALC
26    ALC
27    ALC
28    ALC
29    ALC
30    ALC
31    ALC
32    ALC
33    ALC
34    ALC
35    CTL
36    CTL
37    ALC
38    CTL
39    CTL
40    CTL
41    CTL
42    CTL
43    CTL
44    CTL
45    CTL
46    CTL
47    CTL
48    CTL
49    CTL
50    CTL
51    CTL
52    CTL
53    CTL
Name: GROUP, dtype: object

In [8]:
import mne

def insert_data_eeg(participant_id):
    filePath = rf'C:/Users/DarkK/Documents/data_sci/ds004515/{participant_id}/eeg/{participant_id}_task-ProbabilisticSelection_eeg.set'
    raw = mne.io.read_raw_eeglab(filePath)
    eegData = raw.get_data()
    ch_names = raw.ch_names

    for i in range(len(ch_names)):
        ch_name = ch_names[i]
        ch_data = eegData[i]
        name = ch_name.lower()
        data = sum(ch_data) / len(ch_data)

        if len(name.split(' ')) >= 2:
            name = '_'.join(name.split(' '))
        else:
            name = name

        sqlinsert = f"INSERT INTO eeg_records (participant_id, {name}) VALUES ('{participant_id}', {data})"
        sqlupdate = f"UPDATE eeg_records SET {name} = {data} WHERE participant_id = '{participant_id}'"

        if i == 0:
            cursor.execute(sqlinsert)
            connect.commit()
        else:
            cursor.execute(sqlupdate)
            connect.commit()
    



In [9]:
for i in range(len(participant_id)):
    try:
        insert_data_eeg(participant_id[i])
    except Exception as e:
        print(f"Error inserting data for participant {participant_id[i]}: {e}")
        connect.rollback()

Reading C:\Users\DarkK\Documents\data_sci\ds004515\sub-001\eeg\sub-001_task-ProbabilisticSelection_eeg.fdt


C:\Users\DarkK\AppData\Local\Temp\ipykernel_21812\2009263781.py:5: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filePath)


Reading C:\Users\DarkK\Documents\data_sci\ds004515\sub-002\eeg\sub-002_task-ProbabilisticSelection_eeg.fdt


C:\Users\DarkK\AppData\Local\Temp\ipykernel_21812\2009263781.py:5: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filePath)


Reading C:\Users\DarkK\Documents\data_sci\ds004515\sub-003\eeg\sub-003_task-ProbabilisticSelection_eeg.fdt


...

C:\Users\DarkK\AppData\Local\Temp\ipykernel_21812\2009263781.py:5: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filePath)


Reading C:\Users\DarkK\Documents\data_sci\ds004515\sub-053\eeg\sub-053_task-ProbabilisticSelection_eeg.fdt


C:\Users\DarkK\AppData\Local\Temp\ipykernel_21812\2009263781.py:5: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filePath)


Reading C:\Users\DarkK\Documents\data_sci\ds004515\sub-054\eeg\sub-054_task-ProbabilisticSelection_eeg.fdt


C:\Users\DarkK\AppData\Local\Temp\ipykernel_21812\2009263781.py:5: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filePath)


In [11]:
rawDataSQl = pd.read_sql_query("SELECT * FROM eeg_records", connect)

rawDataSQl

C:\Users\DarkK\AppData\Local\Temp\ipykernel_21812\1044120166.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rawDataSQl = pd.read_sql_query("SELECT * FROM eeg_records", connect)


participant_id       fp1        fz        f3        f7  below_eye  \
0         sub-009 -0.018977 -0.012562 -0.003156 -0.029378  -0.005898   
1         sub-006 -0.005962  0.014713  0.005335  0.022881   0.012168   
2         sub-003 -0.026259 -0.000343 -0.000536 -0.006117  -0.016435   
3         sub-004 -0.013378 -0.004397 -0.006210  0.003812        NaN   
4         sub-011 -0.003784 -0.008590 -0.007245 -0.015636  -0.009475   
5         sub-008 -0.004646  0.002468 -0.006786  0.012283   0.008456   
6         sub-001 -0.006885  0.001531 -0.002528 -0.013197  -0.013694   
7         sub-002  0.006728  0.013835  0.015602  0.006617  -0.002029   
8         sub-005 -0.030890  0.016271 -0.006028 -0.010811        NaN   
9         sub-007 -0.010039 -0.001393 -0.003494 -0.005115  -0.010285   
10        sub-013 -0.007604  0.021150  0.026183 -0.010864  -0.001817   
11        sub-014 -0.021166  0.000999  0.000526 -0.025047  -0.020685   
12        sub-010 -0.003868 -0.002087  0.012339 -0.006504  -0.029840   
13        sub-012  0.005726  0.001772 -0.001047 -0.013619   0.004430   
14        sub-015 -0.004214  0.001439 -0.007950  0.001258  -0.002050   
15        sub-017  0.003264  0.006245  0.015359  0.014401   0.001798   
16        sub-019 -0.023188 -0.006138 -0.024324 -0.030981  -0.004326   
17        sub-031 -0.009776  0.001448 -0.007577  0.004197  -0.013882   
18        sub-018 -0.016266  0.001283 -0.012825 -0.019512  -0.009050   
19        sub-020 -0.018326  0.000844 -0.015240 -0.020218  -0.001486   
20        sub-028 -0.009472 -0.021044 -0.005924 -0.003041  -0.006396   
21        sub-026 -0.020392 -0.017453 -0.011094 -0.008317  -0.018687   
22        sub-021  0.002524  0.003137 -0.007987  0.000470        NaN   
23        sub-016 -0.022919 -0.006373  0.001190 -0.021038  -0.018714   
24        sub-025  0.001850  0.015508  0.009505 -0.005478  -0.000978   
25        sub-022 -0.014094  0.002646 -0.010101 -0.015767        NaN   
26        sub-027 -0.025314 -0.010804 -0.015215 -0.015543  -0.004314   
27        sub-030 -0.014556  0.001812 -0.000285 -0.011779  -0.015137   
28        sub-023 -0.018075  0.000770 -0.008042 -0.008847        NaN   
29        sub-024 -0.009369  0.013676 -0.001488 -0.006310        NaN   
30        sub-029 -0.031649 -0.005814 -0.005986 -0.033177  -0.014575   
31        sub-042 -0.022187 -0.018431 -0.017252 -0.017319  -0.014809   
32        sub-039  0.010320  0.025463  0.011351  0.028762   0.012944   
33        sub-037 -0.022464 -0.022063 -0.017743 -0.027273  -0.018002   
34        sub-032 -0.001825 -0.005650 -0.007588  0.000786  -0.014380   
35        sub-033  0.003262  0.013233  0.001616  0.009194  -0.020096   
36        sub-035 -0.028024 -0.021713 -0.004728 -0.036846  -0.034788   
37        sub-041 -0.008371  0.000245 -0.009281 -0.003000  -0.015570   
38        sub-036  0.001529  0.001408 -0.005508 -0.012362  -0.011743   
39        sub-034  0.002890  0.000224  0.003169  0.016240   0.005839   
40        sub-043 -0.007540  0.011554 -0.003990 -0.001310  -0.007328   
41        sub-038 -0.033660 -0.001765 -0.012161 -0.019336  -0.037943   
42        sub-040 -0.003777  0.001629  0.004618  0.007591  -0.012560   
43        sub-044 -0.005708 -0.003644  0.003865 -0.001842  -0.017365   
44        sub-047 -0.011584 -0.000613 -0.005707 -0.002414  -0.018764   
45        sub-052 -0.020849 -0.017017 -0.017000 -0.020237  -0.024554   
46        sub-045  0.020870  0.023257  0.013297  0.026966   0.004097   
47        sub-050 -0.026579  0.001780 -0.000297  0.009220  -0.031404   
48        sub-046  0.016337  0.007892 -0.005101  0.004846   0.003703   
49        sub-054 -0.007850  0.007753  0.003175  0.010630  -0.006259   
50        sub-048 -0.006969 -0.010642 -0.012793 -0.025560  -0.015865   
51        sub-049 -0.010554 -0.003805 -0.001752  0.007886  -0.013817   
52        sub-051 -0.003496 -0.001667  0.006838  0.009129  -0.005746   
53        sub-053 -0.009593 -0.013785 -0.010336 -0.008232  -0.034691   

         fc5       fc1        c3        t7

In [43]:
mergeDF = pd.merge(participant, rawDataSQl, on='participant_id')
mergeDF.to_csv('D:\eeg_is_burning/data.csv')

In [44]:
eegdata = pd.read_csv('D:\eeg_is_burning/eeg_data.csv')

In [42]:

df_cleaned = eegdata.dropna()

# บันทึกไฟล์ใหม่ (ถ้าต้องการ)
df_cleaned.to_csv('D:\eeg_is_burning/eeg_data.csv', index=False)

print("ข้อมูลที่ลบค่า NaN แล้ว:")
print(df_cleaned)


ข้อมูลที่ลบค่า NaN แล้ว:
    Unnamed: 0 participant_id  Original_ID sex  age GROUP  BDI  AUDIT  \
0            0        sub-001        10600   F   43   CTL   14      1   
1            1        sub-002        10601   F   50   CTL    2      2   
2            2        sub-003        10602   F   22   CTL    2      1   
5            5        sub-006        10605   F   24   CTL    8      3   
6            6        sub-007        10606   M   44   ALC    5      7   
7            7        sub-008        10607   M   38   ALC   42     27   
8            8        sub-009        10608   M   32   ALC    1     11   
9            9        sub-010        10609   F   54   ALC   17      9   
10          10        sub-011        10610   F   37   ALC    5     15   
11          11        sub-012        10611   M   51   ALC    1      7   
12          12        sub-013        10612   F   42   ALC    0      7   
13          13        sub-014        10613   M   33   ALC   24     19   
14          14        sub-

In [89]:
eegdata.loc[(eegdata['AUDIT'] >= 4) & (eegdata['AUDIT'] <= 14 ), 'GROUP'] = 'ALC_early_state'

eegdata.loc[eegdata['AUDIT'] >= 15 , 'GROUP'] = 'ALC_chronic_state'

eegdata.to_csv('eeg_data.csv', index=False)

In [2]:
data2 = pd.read_csv('eeg_data2.csv')

In [3]:
eeg_state = data2.drop(columns=['participant_id','Original_ID','sex','age','EDUCATION'])

In [4]:
eeg_state['GROUP'].value_counts().plot(kind='pie')

<Axes: ylabel='count'>